Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

In [ ]:
dg = DataGroup('../processing/cage.json', load=True)
run = 117
cycle = 1197
# str_query = f'run=={run} and skip==False'
str_query = f'cycle=={cycle} and skip==False'
dg.fileDB.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.fileDB)

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.fileDB['runtype'])
runtype = runtype_list[0]

rt_min = dg.fileDB['runtime'].sum()
u_start = dg.fileDB.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['angle'])[0]
    print(f'Radius: {radius}; Angle: {angle}')
# else:

In [ ]:
# Working with dsp or hit files?

# hit = True 
hit = False #ie working with dsp files

In [ ]:
# Working with calibrated or uncalibrated data?

# cal = True #calibrated data
cal = False #uncalibrated data

In [ ]:
# Working in your own directory or with the CAGE_lh5 directory?

user = False # CAGE_lh5 directory
# user = True # hit filesin my personal directory

In [ ]:
# Get relevant files

lh5_dir = dg.lh5_user_dir if user else dg.lh5_dir
# lh5_dir = '/global/homes/g/gothman/projecta/CAGE_lh5_joule'
print(lh5_dir)

# if hit files
if hit == True:
    file_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
    
else:
    file_list = lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']
    
print(f'lh5_dir: {lh5_dir}')
print(f'file list: {file_list}')

In [ ]:
# Create a dataframe

if (cal == True) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == True) and (hit == False):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')

elif (cal == False) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == False) and (hit == False):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')

else:
    print('dont know what to do here! need to specify if working with calibrated/uncalibrated data, or dsp/hit files')

# df_hit = lh5.load_dfs(file_list, ['trapEmax', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
# print(df_hit.columns)

# dsc = df[['bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)
print(df.columns)



In [ ]:
# lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
# print(lh5_dir)
# hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
# print(hit_list)
# df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEmax_cal', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# # df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# # print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [ ]:
# get raw files for wfs

raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']

print(raw_list)

# don't actually need this, just file list
# raw_df = lh5.load_dfs(raw_list, ['energy'], 'ORSIS3302DecoderForEnergy/raw')



In [ ]:
# select energy typ
# etype = 'trapEmax'
etype = 'energy'

In [ ]:
# use baseline cut
bl_cut_lo, bl_cut_hi = 9000,10500

df_cut = df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()


#create dcr/E 
df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

#create 10-90
df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

#create 50-90
df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

In [ ]:
#creat new DCR
const = 0.0555
df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

In [ ]:
# baseline cut
bl = df['bl']
bl_cut = (bl > 9000) & (bl < 10500)

# print(cut)
print(bl_cut.value_counts())

In [ ]:
##### energy cut 
energy = df[etype]

if cal == True:
    # energy = df['trapEmax_cal']
    energy_cut = (energy > 1450) & (energy < 1470) # 1460 peak
    
else:
    # energy = df_hit['trapEmax']
    # energy_cut = (energy > 2805) & (energy < 2835) # 1460 peak, uncalibratedm trapEmax
    energy_cut = (energy > 2000000) & (energy < 2500000) #where alphas seem to be in run 117, raw energy parameter
    
# print(type(energy))
# print(energy_cut)
print(energy_cut.value_counts())

In [ ]:
cut = bl_cut & energy_cut

In [ ]:
# for debugging

# print(cut)
print(cut.value_counts())

In [ ]:
dsp_config_file = ('./config_dsp.json')
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'dcr_trap'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "Trap (max={trapEmax:.0f})"],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax'], # add hlines and vlines
                          x_lim=(40000, 80000) # x axis range
                         )

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]


In [ ]:
# this is with 6-4-6 trap

In [ ]:
browser.draw_next()
# plt.ylim(2500, 3100)

In [ ]:
dsp_config_file = ('./config_dsp.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz2', 'wf_trap2'], # names of waveforms from dsp config file
                          selection   = cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "Trap (max={trapEmax2:.0f})"],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax2'], # add hlines and vlines
                          x_lim=(40000, 85000) # x axis range
                         )

In [ ]:
browser2.draw_next()
# plt.ylim(2500, 3100)

In [ ]:
df_hit['trapEftp'][483080]

In [ ]:
energy_cut = (trapEmax_cal>355) & (trapEmax_cal<360)
bl_cut = (bl > 8500) & (bl < 10000)
cut2 = energy_cut & bl_cut
dsp_config_file = ('./config_dsp.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_trap'], # names of waveforms from dsp config file
                          selection   = cut2,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "Trap (max={trapEmax:.0f})"],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax', 'tp_0'], # add hlines and vlines
                          x_lim=(15000, 60000) # x axis range
                         )

In [ ]:
browser2.draw_next()

In [ ]:
print(trapEmax_cal[615])
print(energy_cut[615])
# np.nonzero(browser.selection.values)
np.nonzero(energy_cut.values)

In [ ]:
energy_cut = (trapEmax_cal>355) & (trapEmax_cal<360)
bl_cut = (bl > 8500) & (bl < 10000)
cut2 = energy_cut & bl_cut
dsp_config_file = ('./config_dsp.json')
browser3 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub'], # names of waveforms from dsp config file
                          selection   = cut2,           # Apply cut
                          #wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          #legend=['Waveform'],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEmax', 'tp_0'], # add hlines and vlines
                          x_lim=(15000, 60000) # x axis range
                         )

In [ ]:
browser3.draw_next()

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 1000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax_cal'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
# plt.title(f'trapEmax: {runtype} run {run}; {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 6000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(df_cut['trapEmax_cal'], df_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
dlo, dhi, dpb = -0.1, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()

In [ ]:
old_dcr_cut = df_cut.query('trapEmax > 5000 and trapEmax < 15000 and dcr_norm > -0.0515 and dcr_norm < -0.02').copy()
len(old_dcr_cut)

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
e_cut = df_hit.query(f' trapEmax_cal > 348 and trapEmax_cal < 356').copy()
#creat new DCR
const = 0.0555
e_cut['dcr_linoff'] = df_hit['dcr_raw'] + const*df_hit['trapEmax']

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 325, 375, 0.5
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(e_cut['trapEmax_cal'], e_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax_cal', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E 351keV peak: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
# etype = 'trapEmax'

elo, ehi, epb = 325, 375, 0.5
# elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(e_cut['trapEmax_cal'], e_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR 351 keV peak: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
cut2 = (df_cut['dcr_linoff']>50) & (df_cut['dcr_linoff']<200) & (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<12000)
# cut2 = (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<10000)
cut = (df_cut['trapEmax_cal']>1450) & (df_cut['trapEmax_cal']<1470)

In [ ]:
# Now set up browsing waveforms
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_pz', # baseline subtracted waveform
                          #legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          #norm='trapEmax', # normalize by energy
                          selection = cut, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Now set up browsing waveforms
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_pz', # baseline subtracted waveform
                          #legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          #norm='trapEmax', # normalize by energy
                          selection = cut2, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Draw first waveform in file
browser.draw_entry(1)

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
# browser.draw_next()
# browser2.ax = browser.ax
# browser2.fig = browser.fig
# browser2.draw_next()

In [ ]:
# Draw first waveform in file
browser2.draw_entry(5)

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
# browser.draw_next()
# browser2.ax = browser.ax
# browser2.fig = browser.fig
# browser2.draw_next()